Testes usando a Árvore de Decisão

In [ ]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/MyDrive/rdc/tei

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/rdc/tei


In [ ]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

Dados desbalanciados

In [ ]:
print(train_df['class'].value_counts())

class
p    1705396
e    1411549
Name: count, dtype: int64


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

train_df = train_df.drop('id', axis=1)

for col in train_df.columns:
  if col != 'class':
    le.fit(train_df[col])
    train_df[col] = le.transform(train_df[col])

for col in test_df.columns:
    if col != 'id':
      le.fit(test_df[col])
      test_df[col] = le.transform(test_df[col])

In [ ]:
from sklearn.model_selection import train_test_split

X = train_df.drop('class', axis=1)
y = train_df['class']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
train_df

Testando as melhores combinações de hiperparametros com GRID Search

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [8, 11],               # Profundidade máxima da árvore
    'min_samples_leaf': [4, 6]        # Número mínimo de amostras necessárias para estar em uma folha
}

grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42),
                           param_grid,
                           scoring='matthews_corrcoef',
                           cv=10,
                           n_jobs=-1)
grid_search.fit(X_train, y_train)
print(f'Best Parameters: {grid_search.best_params_}')
print(f'Best Score: {grid_search.best_score_}')
print('Best combination')
grid_search.best_estimator_

Best Parameters: {'max_depth': 11, 'min_samples_leaf': 4}
Best Score: 0.8659090761688958
Best combination


DecisionTreeClassifier(max_depth=11, min_samples_leaf=4, random_state=42)

In [ ]:
results_df = pd.DataFrame(grid_search.cv_results_)
sorted_results = results_df.sort_values(by='mean_test_score')


In [ ]:
print(sorted_results)

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
1      49.449500      1.278334         1.870929        0.448073   
0      50.668669      1.659698         1.841327        0.501129   
3      61.687242      2.291907         1.575050        0.526706   
2      62.544339      1.148469         1.498768        0.236140   

  param_max_depth param_min_samples_leaf  \
1               8                      6   
0               8                      4   
3              11                      6   
2              11                      4   

                                     params  split0_test_score  \
1   {'max_depth': 8, 'min_samples_leaf': 6}           0.647984   
0   {'max_depth': 8, 'min_samples_leaf': 4}           0.648001   
3  {'max_depth': 11, 'min_samples_leaf': 6}           0.865031   
2  {'max_depth': 11, 'min_samples_leaf': 4}           0.865102   

   split1_test_score  split2_test_score  split3_test_score  split4_test_score  \
1           0.646857           

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
1      49.449500      1.278334         1.870929        0.448073   
0      50.668669      1.659698         1.841327        0.501129   
3      61.687242      2.291907         1.575050        0.526706   
2      62.544339      1.148469         1.498768        0.236140   

  param_max_depth param_min_samples_leaf  \
1               8                      6   
0               8                      4   
3              11                      6   
2              11                      4   

                                     params  split0_test_score  \
1   {'max_depth': 8, 'min_samples_leaf': 6}           0.647984   
0   {'max_depth': 8, 'min_samples_leaf': 4}           0.648001   
3  {'max_depth': 11, 'min_samples_leaf': 6}           0.865031   
2  {'max_depth': 11, 'min_samples_leaf': 4}           0.865102   

   split1_test_score  split2_test_score  split3_test_score  split4_test_score  \
1           0.646857           0.648903           0.647868           0.645999   
0           0.646865           0.648919           0.647854           0.646014   
3           0.866635           0.865402           0.866582           0.864129   
2           0.866633           0.865516           0.866622           0.864132   

   split5_test_score  split6_test_score  split7_test_score  split8_test_score  \
1           0.648642           0.645722           0.648173           0.649219   
0           0.648626           0.645749           0.648191           0.649217   
3           0.864864           0.867250           0.865861           0.866946   
2           0.864905           0.867195           0.865946           0.866963   

   split9_test_score  mean_test_score  std_test_score  rank_test_score  
1           0.646818         0.647619        0.001149                4  
0           0.646800         0.647623        0.001144                3  
3           0.866069         0.865877        0.000957                2  
2           0.866077         0.865909        0.000938                1  

**Testes diretos no Kaggle**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=3, min_samples_leaf=4, random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=4, random_state=42)

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_val)
mcc_score = matthews_corrcoef(y_val, y_pred)
print(f'MCC Score: {mcc_score}')

MCC Score: 0.30117126956104384


MCC Score: 0.30117126956104384  
Score Kaggle: 0.26161

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=4, min_samples_leaf=4, random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=4, min_samples_leaf=4, random_state=42)

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_val)
mcc_score = matthews_corrcoef(y_val, y_pred)
print(f'MCC Score: {mcc_score}')

MCC Score: 0.3280289977286337


MCC Score: 0.3280289977286337  
Score Kaggle: 0.30454

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=8, min_samples_leaf=5, random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=8, min_samples_leaf=5, random_state=42)

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_val)
mcc_score = matthews_corrcoef(y_val, y_pred)
print(f'MCC Score: {mcc_score}')

MCC Score: 0.6490701472046371


Score Kaggle: 0.12419  
MCC Score: 0.6490701472046371

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=11, min_samples_leaf=4, random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=11, min_samples_leaf=4, random_state=42)

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_val)
mcc_score = matthews_corrcoef(y_val, y_pred)
print(f'MCC Score: {mcc_score}')

MCC Score: 0.8662758300988253


Resultado do kaggle: 0.27684  
MCC Score: 0.8662758300988253

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=13, min_samples_leaf=4, random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=13, min_samples_leaf=4, random_state=42)

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_val)
mcc_score = matthews_corrcoef(y_val, y_pred)
print(f'MCC Score: {mcc_score}')

MCC Score: 0.9255686941642745


MCC Score: 0.9255686941642745  
Score Kaggle: 0.21011

**Validação Cruzada para 1 modelo**

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, matthews_corrcoef
from sklearn.tree import DecisionTreeClassifier

# Definir o modelo
model = DecisionTreeClassifier(max_depth=8, min_samples_leaf=5, min_samples_split=3, random_state=42)

# Definir o scorer como MCC
mcc_scorer = make_scorer(matthews_corrcoef)

# Executar a validação cruzada com MCC
scores = cross_val_score(model, X_train, y_train, cv=10, scoring=mcc_scorer)

# Mostrar as pontuações e a média
print(f'MCC Scores por fold: {scores}')
print(f'Média MCC Score: {scores.mean()}')

MCC Scores por fold: [0.64799923 0.64684105 0.64889714 0.64786828 0.64601259 0.64864163
 0.64575162 0.64817616 0.64921947 0.64681019]
Média MCC Score: 0.6476217353210223


MCC Scores por fold: [0.64799923 0.64684105 0.64889714 0.64786828 0.64601259 0.64864163
 0.64575162 0.64817616 0.64921947 0.64681019]
Média MCC Score: 0.6476217353210223